# Investment Comp Screener

In [1]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
from time import sleep
import quandl
import seaborn as sns
import sys
from scipy.stats import norm
import os
import glob
import re
import seaborn as sns

from statsmodels.distributions.empirical_distribution import ECDF

In [2]:
def calc_ecdf(data):
    #low is good
    ecdf= ECDF(data)
    current_perc = ecdf(data[-1])
    
    return(current_perc)

In [3]:
def color_negative_red(value):
    """
      Colors elements in a dateframe
      green if positive and red if
      negative. Does not color NaN
      values.
      """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

In [113]:
def Z_score(data):
    # not including the current observation
    std = data.iloc[:-1].std()
    mean = data.iloc[:-1].mean()
    Z = (data[-1:] - mean)/std
    return Z
    

## Data Import

In [114]:
path = "holding_companies_data"
arr = [x for x in os.listdir(path) if x.endswith(".csv")]

In [115]:
for f in arr:
    print(f)

2020-11-07_indu c_pris_historik.csv
2020-11-07_hav b_pris_historik.csv
2020-11-07_naxs_pris_historik.csv
2020-11-07_svol b_pris_historik.csv
2020-11-07_lund b_pris_historik.csv
2020-11-07_trac b_pris_historik.csv
2020-11-07_kinv b_pris_historik.csv
2020-11-07_vnv_pris_historik.csv
2020-11-07_bure_pris_historik.csv
2020-11-07_lato b_pris_historik.csv
2020-11-07_cred a_pris_historik.csv
2020-11-07_inve b_pris_historik.csv
2020-11-07_ores_pris_historik.csv


In [116]:
for k in arr:
    if "trac" in k:
        traction = pd.read_csv(os.path.join(path,k))
    if "lund" in k:
        lundbergs = pd.read_csv(os.path.join(path,k))
    if "hav" in k:
        havsfrun = pd.read_csv(os.path.join(path,k))
    if "ores" in k:
        oresund = pd.read_csv(os.path.join(path,k))
    if "indu" in k:
        industri = pd.read_csv(os.path.join(path,k))
    if "svol" in k:
        svolder = pd.read_csv(os.path.join(path,k))
    if "bure" in k:
        bure = pd.read_csv(os.path.join(path,k))
    if "cred" in k:
        creades = pd.read_csv(os.path.join(path,k))
    if "inve" in k:
        investor = pd.read_csv(os.path.join(path,k))
    if "kinv" in k:
        kinnevik = pd.read_csv(os.path.join(path,k))
    if "vnv" in k:
        vnv = pd.read_csv(os.path.join(path,k))
    if "naxs" in k:
        naxs = pd.read_csv(os.path.join(path,k))
    if "lato" in k:
        latour = pd.read_csv(os.path.join(path,k))
    

## Calculate Metrics

In [117]:
look_back = 3*252
ST_look_back = 20


#companies = [traction, lundbergs, havsfrun, oresund, industri,
#             svolder, bure, creades, investor, kinnevik, vnv, naxs,latour]

#companies_names = ['Traction', 'Lundbergsföretagen', 'Havsfrun Investment', 'Öresund', 'Industrivärden',
#            'Svolder', 'Bure', 'Creades', 'Investor', 'Kinnevik', 'VNV Global', 'NAXS','Latour']

companies = [traction, lundbergs, oresund, industri,
             svolder, bure, creades, investor, kinnevik, vnv,latour]

companies_names = ['Traction', 'Lundbergsföretagen', 'Öresund', 'Industrivärden',
             'Svolder', 'Bure', 'Creades', 'Investor', 'Kinnevik', 'VNV Global','Latour']


for comp in companies:
    
    comp['DATUM'] = pd.to_datetime(comp['DATUM'])
    comp.set_index(['DATUM'], inplace=True, drop=True)
    comp['R/P']=comp['PRIS']/comp['BERÄKNAT_SUBSTANSVÄRDE'] - 1
    



In [125]:
#high DSSP and HIDI is good

AVG_DISC = pd.DataFrame(latour['PRIS'].copy())
AVG_DISC['PRIS'] = 0
AVG_DISC.columns = ['AVG_DISC']

for comp in companies:

    AVG_DISC['AVG_DISC'] = AVG_DISC['AVG_DISC'] + comp['R/P']

AVG_DISC = AVG_DISC/len(companies)

for comp in companies:
    comp['SPREAD'] = comp['R/P'] - AVG_DISC['AVG_DISC']
    #comp['DSSP'] = -comp['SPREAD'].rolling(look_back).apply(lambda x: calc_ecdf(x),raw=False) + 1
    comp['DSSP'] = comp['SPREAD'].rolling(look_back).apply(lambda x: Z_score(x),raw=False) 
    #comp['HIDI'] = -comp['R/P'].rolling(look_back).apply(lambda x: calc_ecdf(x),raw=False) + 1
    comp['HIDI'] = comp['R/P'].rolling(look_back).apply(lambda x: Z_score(x),raw=False)
    comp['MOM'] = (comp['PRIS'].pct_change(252) + comp['PRIS'].pct_change(126) + comp['PRIS'].pct_change(63))/3
    comp['NAV MOM'] = (comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(252) + comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(126) + comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(63))/3
    comp['ST'] = comp['PRIS'].pct_change(ST_look_back)
    
    

In [126]:
INV_COMP = None

for comp in companies:
    
    INV_COMP = pd.concat([INV_COMP, comp.tail(1)], axis = 0)

INV_COMP['Company'] = companies_names
INV_COMP.set_index(['Company'], inplace=True, drop=True)

INV_COMP['Company'] = companies_names


In [127]:
INV_COMP['VRANK'] = (INV_COMP['R/P'].rank() + INV_COMP['DSSP'].rank(ascending = False) +
 INV_COMP['HIDI'].rank(ascending = False)).rank()



INV_COMP['MRANK'] = (INV_COMP['MOM'].rank(ascending = False)+INV_COMP['NAV MOM'].rank(ascending = False)).rank()

INV_COMP['COMB RANK'] = (INV_COMP['VRANK'] + INV_COMP['MRANK']).rank()


compact = ['R/P', 'DSSP', 'HIDI', 'MOM', 'NAV MOM', 'VRANK', 'MRANK', 'COMB RANK', 'ST']

INV_COMP = INV_COMP[compact]

INV_COMP['R/P']=-INV_COMP['R/P']

In [128]:
average = pd.DataFrame(INV_COMP.mean(axis = 0)).transpose()
average.index = ['Average']
INV_COMP = pd.concat([INV_COMP, average], axis = 0)

## Display Results

In [123]:
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]

cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(INV_COMP.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(INV_COMP.columns), as_cmap=True)

INV_COMP_VIEW = (INV_COMP.style
    .applymap(color_negative_red, subset=['R/P', 'MOM', 'NAV MOM'])
    .background_gradient(axis = 0 ,subset = ['DSSP', 'HIDI'],cmap=cm_high_good)
    .background_gradient(axis = 0 ,subset = ['VRANK', 'MRANK','COMB RANK'],cmap=cm_low_good)
    #.set_caption('Performance Metrics')
    .format({'R/P': "{:.1%}", 'DSSP': "{:.0%}", 'HIDI': "{:.0%}", 'MOM':"{:.1%}", 'NAV MOM': "{:.1%}",
            'Sortino': "{:.2f}", 'MAR': "{:.2f}", 'VRANK': "{:.0f}", 'MRANK': "{:.0f}", 'COMB RANK': "{:.0f}",'ST':"{:.1%}"})
    .set_table_styles(styles))

In [124]:
INV_COMP_VIEW

,R/P,DSSP,HIDI,MOM,NAV MOM,VRANK,MRANK,COMB RANK,ST
Traction,6.9%,100%,-19%,0.9%,7.3%,4,11,11,1.2%
Lundbergsföretagen,-4.9%,100%,33%,6.3%,13.8%,7,6,7,-1.6%
Öresund,-2.2%,100%,-17%,12.9%,13.1%,8,6,10,-0.3%
Industrivärden,3.4%,95%,245%,17.8%,8.1%,2,8,4,2.9%
Svolder,-11.4%,62%,331%,55.5%,24.9%,6,3,2,2.8%
Bure,-29.1%,8%,232%,46.0%,15.4%,10,4,8,0.1%
Creades,-89.0%,0%,307%,107.2%,33.7%,11,2,5,8.1%
Investor,18.0%,100%,95%,16.1%,11.0%,1,8,3,1.5%
Kinnevik,-4.1%,49%,315%,69.5%,34.1%,6,2,1,15.5%
VNV Global,13.1%,84%,96%,25.4%,-0.9%,3,10,6,0.9%


In [100]:
comp[-1:]

,PRIS,SUBSTANSVÄRDE,BERÄKNAT_SUBSTANSVÄRDE,R/P,SPREAD,DSSP,MOM,NAV MOM,ST
DATUM,,,,,,,,,
2020-11-06,235.0,143.0,144.26,0.629003,0.481554,0.002646,0.484112,0.112049,0.113744


In [110]:
INV_COMP

,R/P,DSSP,HIDI,MOM,NAV MOM,VRANK,MRANK,COMB RANK,ST
Traction,0.068811,0.998677,0.193418,0.008534,0.073344,1.0,11.0,7.0,0.011765
Lundbergsföretagen,-0.048934,0.998677,-0.325385,0.062639,0.138056,4.0,6.5,3.5,-0.015568
Öresund,-0.022498,0.997354,0.165224,0.129173,0.131348,3.0,6.5,2.0,-0.003145
Industrivärden,0.033695,0.953704,-2.439638,0.178111,0.080537,6.0,8.5,9.0,0.028866
Svolder,-0.113895,0.615079,-3.288411,0.554623,0.249478,9.0,3.0,7.0,0.028470
Bure,-0.290890,0.080688,-2.307442,0.460466,0.153682,8.0,4.0,7.0,0.001436
Creades,-0.890312,0.003968,-3.048142,1.071974,0.336564,10.0,1.5,5.0,0.080522
Investor,0.180232,0.997354,-0.951221,0.161494,0.110327,2.0,8.5,3.5,0.015194
Kinnevik,-0.040535,0.492063,-3.127937,0.694582,0.341392,7.0,1.5,1.0,0.155440
VNV Global,0.131393,0.841270,-0.955546,0.253596,-0.009062,5.0,10.0,10.0,0.008963


In [112]:
comp.iloc[:-1] 

,PRIS,SUBSTANSVÄRDE,BERÄKNAT_SUBSTANSVÄRDE,R/P,SPREAD,DSSP,MOM,NAV MOM,ST,HIDI
DATUM,,,,,,,,,,
2014-03-03,188.5,212.0,0.00,inf,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-04,188.5,212.0,0.00,inf,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-05,187.6,212.0,0.00,inf,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-06,186.5,212.0,0.00,inf,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-07,184.8,212.0,0.00,inf,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-10-30,207.8,140.0,131.78,0.576871,0.478578,0.000000,0.364994,0.011301,-0.007641,-3.273534
2020-11-02,208.8,140.0,132.52,0.575611,0.472342,0.001323,0.355242,0.011412,-0.004766,-3.237212
2020-11-03,214.6,140.0,133.61,0.606167,0.484672,0.000000,0.375965,0.020733,0.033719,-3.424056
